# 01 — Exploratory Data Analysis: OHLCV Price Data

**Goal:** Understand the dataset before building any ML models.

We'll cover:
1. Loading data from our silver layer (parquet) and DuckDB
2. Shape, dtypes, missing values
3. Price distributions and summary statistics
4. Time-series plots per symbol
5. Daily returns — distribution and volatility
6. Correlation between stocks
7. Volume analysis
8. Key takeaways for feature engineering

In [2]:
import sys
sys.path.insert(0, "..")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import duckdb

sns.set_theme(style="whitegrid", palette="muted")
plt.rcParams["figure.figsize"] = (14, 5)
plt.rcParams["figure.dpi"] = 100

print("Libraries loaded.")

Libraries loaded.


## 1. Load Data

Two ways to access our data — direct parquet read, or via DuckDB SQL.
We'll use both so you get comfortable with each approach.

In [3]:
# Method 1: Direct parquet read
ohlcv = pd.read_parquet("../data/silver/ohlcv_daily.parquet")
dim_assets = pd.read_parquet("../data/silver/dim_assets.parquet")

print(f"OHLCV rows: {len(ohlcv):,}")
print(f"Assets: {len(dim_assets)}")
print(f"Symbols: {sorted(ohlcv['symbol'].unique())}")
print(f"Date range: {ohlcv['ts'].min()} to {ohlcv['ts'].max()}")

OHLCV rows: 89,719
Assets: 10
Symbols: ['AAPL', 'AMZN', 'GOOGL', 'JNJ', 'JPM', 'META', 'MSFT', 'NVDA', 'TSLA', 'XOM']
Date range: 1970-01-02 00:00:00 to 2026-02-06 00:00:00


In [4]:
# Method 2: DuckDB SQL — useful for quick aggregations
con = duckdb.connect("../duckdb/market.duckdb", read_only=True)

con.sql("""
    SELECT symbol, 
           COUNT(*) as trading_days,
           MIN(ts) as first_date,
           MAX(ts) as last_date
    FROM ohlcv_daily
    GROUP BY symbol
    ORDER BY symbol
""").show()

con.close()

┌─────────┬──────────────┬─────────────────────┬─────────────────────┐
│ symbol  │ trading_days │     first_date      │      last_date      │
│ varchar │    int64     │    timestamp_ns     │    timestamp_ns     │
├─────────┼──────────────┼─────────────────────┼─────────────────────┤
│ AAPL    │        10435 │ 1984-09-07 00:00:00 │ 2026-02-06 00:00:00 │
│ AMZN    │         7222 │ 1997-05-16 00:00:00 │ 2026-02-06 00:00:00 │
│ GOOGL   │         5402 │ 2004-08-19 00:00:00 │ 2026-02-06 00:00:00 │
│ JNJ     │        14143 │ 1970-01-02 00:00:00 │ 2026-02-06 00:00:00 │
│ JPM     │        14143 │ 1970-01-02 00:00:00 │ 2026-02-06 00:00:00 │
│ META    │         3450 │ 2012-05-18 00:00:00 │ 2026-02-06 00:00:00 │
│ MSFT    │        10052 │ 1986-03-13 00:00:00 │ 2026-02-06 00:00:00 │
│ NVDA    │         6802 │ 1999-01-22 00:00:00 │ 2026-02-06 00:00:00 │
│ TSLA    │         3927 │ 2010-06-28 00:00:00 │ 2026-02-06 00:00:00 │
│ XOM     │        14143 │ 1970-01-02 00:00:00 │ 2026-02-06 00:00:00 │
├─────

## 2. Data Quality Check

Before any analysis — always check for nulls, dtypes, and anomalies.

In [5]:
print("=== Data Types ===")
print(ohlcv.dtypes)
print()

print("=== Missing Values ===")
print(ohlcv.isnull().sum())
print()

print("=== Duplicates ===")
dupes = ohlcv.duplicated(subset=["source", "symbol", "ts"]).sum()
print(f"Duplicate rows (source, symbol, ts): {dupes}")
print()

print("=== Sample Rows ===")
ohlcv.head(10)

=== Data Types ===
source                      object
symbol                      object
ts                  datetime64[ns]
open                       float64
high                       float64
low                        float64
close                      float64
volume                     float64
ingested_at    datetime64[us, UTC]
dtype: object

=== Missing Values ===
source         0
symbol         0
ts             0
open           0
high           0
low            0
close          0
volume         0
ingested_at    0
dtype: int64

=== Duplicates ===
Duplicate rows (source, symbol, ts): 0

=== Sample Rows ===


,source,symbol,ts,open,high,low,close,volume,ingested_at
0,stooq,AAPL,1984-09-07,0.099265,0.100484,0.098067,0.099265,99149603.0,2026-02-08 23:08:15.320637+00:00
1,stooq,AAPL,1984-09-10,0.099265,0.099570,0.096879,0.098676,76956267.0,2026-02-08 23:08:15.320637+00:00
2,stooq,AAPL,1984-09-11,0.099570,0.102271,0.099570,0.100484,181467446.0,2026-02-08 23:08:15.320637+00:00
3,stooq,AAPL,1984-09-12,0.100484,0.101071,0.097458,0.097458,158527291.0,2026-02-08 23:08:15.320637+00:00
4,stooq,AAPL,1984-09-13,0.102880,0.103173,0.102880,0.102880,246900394.0,2026-02-08 23:08:15.320637+00:00
5,stooq,AAPL,1984-09-14,0.103173,0.106760,0.103173,0.104363,293981051.0,2026-02-08 23:08:15.320637+00:00
6,stooq,AAPL,1984-09-17,0.107073,0.108567,0.107073,0.107073,229535149.0,2026-02-08 23:08:15.320637+00:00
7,stooq,AAPL,1984-09-18,0.107073,0.107957,0.103173,0.103173,115874598.0,2026-02-08 23:08:15.320637+00:00
8,stooq,AAPL,1984-09-19,0.103173,0.104363,0.101071,0.101071,126571111.0,2026-02-08 23:08:15.320637+00:00
9,stooq,AAPL,1984-09-20,0.101368,0.102271,0.101368,0.101368,78796873.0,2026-02-08 23:08:15.320637+00:00


In [ ]:
# Check for zero or negative prices (data quality red flags)
for col in ["open", "high", "low", "close"]:
    bad = (ohlcv[col] <= 0).sum()
    if bad > 0:
        print(f"WARNING: {bad} rows with {col} <= 0")

# Check OHLC consistency: high >= low, high >= open, high >= close
inconsistent = ohlcv[ohlcv["high"] < ohlcv["low"]]
print(f"Rows where high < low: {len(inconsistent)}")

# Check for zero volume days
zero_vol = ohlcv[ohlcv["volume"] == 0]
print(f"Zero-volume days: {len(zero_vol)}")

## 3. Summary Statistics

Get a feel for the scale and distribution of each stock.

In [ ]:
summary = ohlcv.groupby("symbol")["close"].describe().round(2)
summary

## 4. Price Time Series

Visualize closing prices over time. This immediately reveals trends, crashes, and regime changes.

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))

for sym in sorted(ohlcv["symbol"].unique()):
    subset = ohlcv[ohlcv["symbol"] == sym].sort_values("ts")
    ax.plot(subset["ts"], subset["close"], label=sym, linewidth=1)

ax.set_title("Daily Closing Prices")
ax.set_xlabel("Date")
ax.set_ylabel("Price (USD)")
ax.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()

In [ ]:
# Normalized prices (base 100) — easier to compare performance
fig, ax = plt.subplots(figsize=(16, 6))

for sym in sorted(ohlcv["symbol"].unique()):
    subset = ohlcv[ohlcv["symbol"] == sym].sort_values("ts")
    normalized = (subset["close"] / subset["close"].iloc[0]) * 100
    ax.plot(subset["ts"], normalized, label=sym, linewidth=1)

ax.set_title("Normalized Closing Prices (Base = 100)")
ax.set_xlabel("Date")
ax.set_ylabel("Indexed Price")
ax.axhline(y=100, color="gray", linestyle="--", alpha=0.5)
ax.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()

## 5. Daily Returns Analysis

Returns (percentage change) are more useful for ML than raw prices:
- Prices are non-stationary (they trend up/down)
- Returns are roughly stationary and comparable across stocks

This is a **key concept** for financial ML.

In [ ]:
# Calculate daily returns per symbol
ohlcv = ohlcv.sort_values(["symbol", "ts"])
ohlcv["daily_return"] = ohlcv.groupby("symbol")["close"].pct_change()

# Drop first row per symbol (NaN return)
returns_df = ohlcv.dropna(subset=["daily_return"])

returns_df.groupby("symbol")["daily_return"].describe().round(4)

In [ ]:
# Distribution of daily returns — are they normal?
fig, axes = plt.subplots(2, 5, figsize=(18, 7), sharex=True)

for ax, sym in zip(axes.flatten(), sorted(returns_df["symbol"].unique())):
    data = returns_df[returns_df["symbol"] == sym]["daily_return"]
    ax.hist(data, bins=80, edgecolor="none", alpha=0.7)
    ax.axvline(0, color="red", linestyle="--", alpha=0.5)
    ax.set_title(sym, fontsize=11)
    ax.set_xlim(-0.15, 0.15)

fig.suptitle("Distribution of Daily Returns", fontsize=14)
fig.supxlabel("Daily Return")
fig.supylabel("Frequency")
plt.tight_layout()
plt.show()

In [ ]:
# Rolling 30-day volatility (std of returns)
fig, ax = plt.subplots(figsize=(16, 6))

for sym in sorted(returns_df["symbol"].unique()):
    subset = returns_df[returns_df["symbol"] == sym].sort_values("ts")
    rolling_vol = subset["daily_return"].rolling(30).std() * np.sqrt(252)  # annualized
    ax.plot(subset["ts"], rolling_vol, label=sym, linewidth=1)

ax.set_title("30-Day Rolling Annualized Volatility")
ax.set_xlabel("Date")
ax.set_ylabel("Volatility")
ax.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.tight_layout()
plt.show()

## 6. Correlation Analysis

How do these stocks move relative to each other? High correlation means they tend to move together.
This matters for:
- Portfolio diversification
- Feature selection (correlated features may be redundant)

In [ ]:
# Pivot returns into wide format: each column = one stock's daily return
returns_wide = returns_df.pivot_table(
    index="ts", columns="symbol", values="daily_return"
)

corr = returns_wide.corr()

fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(
    corr,
    annot=True,
    fmt=".2f",
    cmap="RdYlBu_r",
    center=0,
    square=True,
    linewidths=0.5,
    ax=ax,
)
ax.set_title("Correlation Matrix — Daily Returns")
plt.tight_layout()
plt.show()

## 7. Volume Analysis

Volume confirms price moves — a price increase on high volume is more significant than on low volume.

In [ ]:
# Average daily volume by symbol
avg_vol = ohlcv.groupby("symbol")["volume"].mean().sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(10, 5))
avg_vol.plot(kind="bar", ax=ax, edgecolor="none")
ax.set_title("Average Daily Trading Volume")
ax.set_ylabel("Shares")
ax.ticklabel_format(axis="y", style="plain")
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()

In [ ]:
# Volume-price relationship for one stock (AAPL)
aapl = ohlcv[ohlcv["symbol"] == "AAPL"].sort_values("ts").copy()

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 8), sharex=True)

ax1.plot(aapl["ts"], aapl["close"], color="steelblue", linewidth=1)
ax1.set_ylabel("Close Price")
ax1.set_title("AAPL — Price vs Volume")

ax2.bar(aapl["ts"], aapl["volume"], color="gray", alpha=0.6, width=1)
ax2.set_ylabel("Volume")
ax2.set_xlabel("Date")

plt.tight_layout()
plt.show()

## 8. Candlestick-style OHLC View

Quick look at the spread between open/high/low/close — this is what traders look at.

In [ ]:
# Intraday range as a % of close — how much does the stock move within a day?
ohlcv["intraday_range_pct"] = (ohlcv["high"] - ohlcv["low"]) / ohlcv["close"] * 100

range_stats = ohlcv.groupby("symbol")["intraday_range_pct"].agg(["mean", "median", "max"]).round(2)
range_stats.columns = ["mean_%", "median_%", "max_%"]
range_stats.sort_values("mean_%", ascending=False)

## 9. Key Takeaways

Run this after examining the outputs above — summarize what you observe.

Things to note:
- **Which stocks are most volatile?** (TSLA, NVDA likely)
- **Are returns normally distributed?** (Probably fat-tailed)
- **Which stocks are most correlated?** (Tech cluster likely)
- **Any data quality issues?** (Missing days, zero volume, etc.)
- **Date range coverage** — is it sufficient for training?

These observations will guide our feature engineering in the next notebook.

In [ ]:
# Quick summary for reference
print("=== Dataset Summary ===")
print(f"Symbols: {sorted(ohlcv['symbol'].unique())}")
print(f"Total rows: {len(ohlcv):,}")
print(f"Date range: {ohlcv['ts'].min()} to {ohlcv['ts'].max()}")
print(f"Missing values: {ohlcv[['open','high','low','close','volume']].isnull().sum().sum()}")
print(f"\nMost volatile (avg daily return std):")
print(returns_df.groupby('symbol')['daily_return'].std().sort_values(ascending=False).head(3).round(4).to_string())
print(f"\nHighest avg volume:")
print(avg_vol.head(3).apply(lambda x: f"{x:,.0f}").to_string())

---

**Next:** `02_feature_engineering.ipynb` — Build technical indicators (RSI, MACD, moving averages, Bollinger Bands) and prepare features for ML models.